In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
movies_file_path = "/Users/evavashisth/Desktop/CSC👩🏻‍💻/Personal Projects/movie_rcmnd/data/tmdb_5000_movies.csv"
credits_file_path = "/Users/evavashisth/Desktop/CSC👩🏻‍💻/Personal Projects/movie_rcmnd/data/tmdb_5000_credits.csv"

movies = pd.read_csv(movies_file_path)
credits = pd.read_csv(credits_file_path)

In [ ]:
movies.head(2)

In [ ]:
credits.head(2)

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
movies.columns

In [ ]:
movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies['tags'] = movies['overview'] + ' ' + movies['genres'] + ' ' + movies['keywords'] + ' ' + movies['cast'] + ' ' + movies['crew']
movies.head()


In [ ]:
first_row_tags = movies.iloc[0]['tags']

In [ ]:
new_df = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']].copy()


In [ ]:
new_df.head()

In [ ]:
movies.isnull().sum()


In [ ]:
movies.dropna(inplace=True)
movies.shape
movies.duplicated().sum()
movies.head(2)

In [ ]:
movies.iloc[0]['genres']

In [ ]:
import ast
def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert_cast(text):
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            l.append(i['name'])
        counter +=1
    return l

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)
movies.head(2)

In [ ]:
def fetch_director(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break

    return l

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head(2)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

In [ ]:
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(' ', ''))
    return l

In [ ]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)
movies.head()

In [ ]:
movies['tags'] = movies ['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head()

In [ ]:
new_df = movies[['id','title', 'tags']]
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()


In [ ]:
new_df.iloc[0]['tags']

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

In [ ]:
import nltk
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
        
    return " ".join(l)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stems)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words= 'english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)
similarity

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Spider-Man')

In [ ]:
import pickle

directory = 'pkls'
if not os.path.exists(directory):
    os.makedirs(directory)

pickle.dump(new_df, open('pkls/movie_list.pkl', 'wb'))
pickle.dump(similarity, open('pkls/similarity.pkl', 'wb'))
